# Import Libraries

In [1]:
import torch
def format_pytorch_version(version):
  return version.split('+')[0]
TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
def format_cuda_version(version):
  return 'cu' + version.replace('.', '')
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install pyg
import os.path as osp
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 15.3 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 44.7 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.3 MB 20.8 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 747 kB 26.6 MB/s 
     |████████████████████████████████| 370 kB 29.2 MB/s 
     |████████████████████████████████| 482 kB 52.1 MB/s 
     |████████████████████████████████| 41 kB 546 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=45398b1c519a35f2735d6e9cfba16daa02ea26126c9e6e5df3f051206ce8d41d
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully bui

# Custom Gat Layer

In [ ]:
from typing import Optional, Tuple, Union, Any

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Parameter
from torch_sparse import SparseTensor, set_diag

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import (Adj, NoneType, OptPairTensor, OptTensor,
                                    Size)
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax

from torch_geometric.nn.inits import glorot, zeros


class CustomGATConv(MessagePassing):

    _alpha: OptTensor

    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        heads: int = 1,
        concat: bool = True,
        negative_slope: float = 0.2,
        dropout: float = 0.0,
        add_self_loops: bool = True,
        edge_dim: Optional[int] = None,
        fill_value: Union[float, Tensor, str] = 'mean',
        bias: bool = True,
        custom_func: Optional[Any] = lambda x,y: (x,y),
        **kwargs,
    ):
        kwargs.setdefault('aggr', 'add')
        super().__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.add_self_loops = add_self_loops
        self.edge_dim = edge_dim
        self.fill_value = fill_value
        self.custom_func = custom_func

        # In case we are operating in bipartite graphs, we apply separate
        # transformations 'lin_src' and 'lin_dst' to source and target nodes:
        if isinstance(in_channels, int):
            self.lin_src = Linear(in_channels, heads * out_channels,
                                  bias=False, weight_initializer='glorot')
            self.lin_dst = self.lin_src
        else:
            self.lin_src = Linear(in_channels[0], heads * out_channels, False,
                                  weight_initializer='glorot')
            self.lin_dst = Linear(in_channels[1], heads * out_channels, False,
                                  weight_initializer='glorot')

        # The learnable parameters to compute attention coefficients:
        self.att_src = Parameter(torch.Tensor(1, heads, out_channels))
        self.att_dst = Parameter(torch.Tensor(1, heads, out_channels))

        if edge_dim is not None:
            self.lin_edge = Linear(edge_dim, heads * out_channels, bias=False,
                                   weight_initializer='glorot')
            self.att_edge = Parameter(torch.Tensor(1, heads, out_channels))
        else:
            self.lin_edge = None
            self.register_parameter('att_edge', None)

        if bias and concat:
            self.bias = Parameter(torch.Tensor(heads * out_channels))
        elif bias and not concat:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self._alpha = None

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_src.reset_parameters()
        self.lin_dst.reset_parameters()
        if self.lin_edge is not None:
            self.lin_edge.reset_parameters()
        glorot(self.att_src)
        glorot(self.att_dst)
        glorot(self.att_edge)
        zeros(self.bias)

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                edge_attr: OptTensor = None, size: Size = None,
                return_attention_weights=None):


        H, C = self.heads, self.out_channels

        # We first transform the input node features. If a tuple is passed, we
        # transform source and target node features via separate weights:
        if isinstance(x, Tensor):
            assert x.dim() == 2, "Static graphs not supported in 'GATConv'"
            x_src = x_dst = self.lin_src(x).view(-1, H, C)
        else:  # Tuple of source and target node features:
            x_src, x_dst = x
            assert x_src.dim() == 2, "Static graphs not supported in 'GATConv'"
            x_src = self.lin_src(x_src).view(-1, H, C)
            if x_dst is not None:
                x_dst = self.lin_dst(x_dst).view(-1, H, C)

        x = (x_src, x_dst)

        # Next, we compute node-level attention coefficients, both for source
        # and target nodes (if present):
        alpha_src = (x_src * self.att_src).sum(dim=-1)
        alpha_dst = None if x_dst is None else (x_dst * self.att_dst).sum(-1)
        alpha = self.custom_func(alpha_src, alpha_dst)

        if self.add_self_loops:
            if isinstance(edge_index, Tensor):
                # We only want to add self-loops for nodes that appear both as
                # source and target nodes:
                num_nodes = x_src.size(0)
                if x_dst is not None:
                    num_nodes = min(num_nodes, x_dst.size(0))
                num_nodes = min(size) if size is not None else num_nodes
                edge_index, edge_attr = remove_self_loops(
                    edge_index, edge_attr)
                edge_index, edge_attr = add_self_loops(
                    edge_index, edge_attr, fill_value=self.fill_value,
                    num_nodes=num_nodes)
            elif isinstance(edge_index, SparseTensor):
                if self.edge_dim is None:
                    edge_index = set_diag(edge_index)
                else:
                    raise NotImplementedError(
                        "The usage of 'edge_attr' and 'add_self_loops' "
                        "simultaneously is currently not yet supported for "
                        "'edge_index' in a 'SparseTensor' form")

        # propagate_type: (x: OptPairTensor, alpha: OptPairTensor, edge_attr: OptTensor)  # noqa
        out = self.propagate(edge_index, x=x, alpha=alpha, edge_attr=edge_attr,
                             size=size)

        alpha = self._alpha
        assert alpha is not None
        self._alpha = None

        if self.concat:
            out = out.view(-1, self.heads * self.out_channels)
        else:
            out = out.mean(dim=1)

        if self.bias is not None:
            out += self.bias

        if isinstance(return_attention_weights, bool):
            if isinstance(edge_index, Tensor):
                return out, (edge_index, alpha)
            elif isinstance(edge_index, SparseTensor):
                return out, edge_index.set_value(alpha, layout='coo')
        else:
            return out

    def message(self, x_j: Tensor, alpha_j: Tensor, alpha_i: OptTensor,
                edge_attr: OptTensor, index: Tensor, ptr: OptTensor,
                size_i: Optional[int]) -> Tensor:
        # Given edge-level attention coefficients for source and target nodes,
        # we simply need to sum them up to "emulate" concatenation:
        alpha = alpha_j if alpha_i is None else alpha_j + alpha_i

        if edge_attr is not None:
            if edge_attr.dim() == 1:
                edge_attr = edge_attr.view(-1, 1)
            assert self.lin_edge is not None
            edge_attr = self.lin_edge(edge_attr)
            edge_attr = edge_attr.view(-1, self.heads, self.out_channels)
            alpha_edge = (edge_attr * self.att_edge).sum(dim=-1)
            alpha = alpha + alpha_edge

        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, index, ptr, size_i)
        self._alpha = alpha  # Save for later use.
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        return x_j * alpha.unsqueeze(-1)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, heads={self.heads})')

In [ ]:
minFunc = lambda a, b : torch.minimum(a, b)
maxFunc = lambda a, b : torch.maximum(a, b)
sumFunc = lambda a, b : a + b
meanFunc = lambda a, b : (a + b) / 2
hadamardFunc = lambda a, b : a * b

# Classification task on Cora dataset with CustomGATConv(minFunc)

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = CustomGATConv(in_channels, 8, heads=8, dropout=0.6, custom_func=minFunc)
        self.conv2 = CustomGATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6, custom_func=minFunc)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.4000, Val: 0.2500, Test: 0.2540
Epoch: 002, Train: 0.4429, Val: 0.2680, Test: 0.2870
Epoch: 003, Train: 0.5643, Val: 0.3440, Test: 0.3470
Epoch: 004, Train: 0.5929, Val: 0.3680, Test: 0.3800
Epoch: 005, Train: 0.6786, Val: 0.4120, Test: 0.4180
Epoch: 006, Train: 0.7571, Val: 0.4980, Test: 0.5030
Epoch: 007, Train: 0.8071, Val: 0.5920, Test: 0.5800
Epoch: 008, Train: 0.8571, Val: 0.6720, Test: 0.6380
Epoch: 009, Train: 0.8786, Val: 0.7100, Test: 0.6850
Epoch: 010, Train: 0.9000, Val: 0.7540, Test: 0.7370
Epoch: 011, Train: 0.9071, Val: 0.7680, Test: 0.7500
Epoch: 012, Train: 0.9143, Val: 0.7800, Test: 0.7670
Epoch: 013, Train: 0.9143, Val: 0.7880, Test: 0.7810
Epoch: 014, Train: 0.9214, Val: 0.7860, Test: 0.7780
Epoch: 015, Train: 0.9500, Val: 0.7820, Test: 0.7820
Epoch: 016, Train: 0.9571, Val: 0.7840, Test: 0.7810
Epoch: 017, Train: 0.9500, Val: 0.7820, Test: 0.7890
Epoch: 018, Train: 0.9571, Val: 0.7800, Test: 0.7860
Epoch: 019, Train: 0.9571, Val: 0.7760, Test: 

# Classification task on Cora dataset with CustomGATConv(maxFunc)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = CustomGATConv(in_channels, 8, heads=8, dropout=0.6, custom_func=maxFunc)
        self.conv2 = CustomGATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6, custom_func=minFunc)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2929, Val: 0.2160, Test: 0.2210
Epoch: 002, Train: 0.3857, Val: 0.2500, Test: 0.2680
Epoch: 003, Train: 0.4500, Val: 0.2840, Test: 0.2940
Epoch: 004, Train: 0.5000, Val: 0.2840, Test: 0.3150
Epoch: 005, Train: 0.6429, Val: 0.4020, Test: 0.4120
Epoch: 006, Train: 0.6857, Val: 0.4400, Test: 0.4410
Epoch: 007, Train: 0.7214, Val: 0.4700, Test: 0.4610
Epoch: 008, Train: 0.7500, Val: 0.5160, Test: 0.5090
Epoch: 009, Train: 0.8357, Val: 0.6040, Test: 0.6010
Epoch: 010, Train: 0.8929, Val: 0.6540, Test: 0.6480
Epoch: 011, Train: 0.9214, Val: 0.6940, Test: 0.6830
Epoch: 012, Train: 0.9286, Val: 0.7580, Test: 0.7370
Epoch: 013, Train: 0.9357, Val: 0.7720, Test: 0.7660
Epoch: 014, Train: 0.9500, Val: 0.7860, Test: 0.7860
Epoch: 015, Train: 0.9357, Val: 0.7860, Test: 0.8020
Epoch: 016, Train: 0.9429, Val: 0.7960, Test: 0.8110
Epoch: 017, Train: 0.9429, Val: 0.7880, Test: 0.7980
Epoch: 018, Train: 0.9571, Val: 0.7840, Test: 0.7870
Epoch: 019, Train: 0.9429, Val: 0.7740, Test: 

# Classification task on Cora dataset with CustomGATConv(sumFunc)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = CustomGATConv(in_channels, 8, heads=8, dropout=0.6, custom_func=sumFunc)
        self.conv2 = CustomGATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6, custom_func=sumFunc)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2000, Val: 0.2660, Test: 0.2310
Epoch: 002, Train: 0.3000, Val: 0.3140, Test: 0.2840
Epoch: 003, Train: 0.4286, Val: 0.3420, Test: 0.3290
Epoch: 004, Train: 0.5500, Val: 0.3820, Test: 0.3800
Epoch: 005, Train: 0.5143, Val: 0.3600, Test: 0.3540
Epoch: 006, Train: 0.5000, Val: 0.3120, Test: 0.2970
Epoch: 007, Train: 0.5071, Val: 0.3120, Test: 0.2870
Epoch: 008, Train: 0.5500, Val: 0.3380, Test: 0.3040
Epoch: 009, Train: 0.6357, Val: 0.4020, Test: 0.3940
Epoch: 010, Train: 0.7071, Val: 0.4600, Test: 0.4590
Epoch: 011, Train: 0.8000, Val: 0.5380, Test: 0.5370
Epoch: 012, Train: 0.8714, Val: 0.6060, Test: 0.6090
Epoch: 013, Train: 0.9000, Val: 0.6660, Test: 0.6600
Epoch: 014, Train: 0.9071, Val: 0.7120, Test: 0.7110
Epoch: 015, Train: 0.9143, Val: 0.7120, Test: 0.6990
Epoch: 016, Train: 0.8929, Val: 0.6840, Test: 0.6920
Epoch: 017, Train: 0.8929, Val: 0.6780, Test: 0.6910
Epoch: 018, Train: 0.8929, Val: 0.6760, Test: 0.7130
Epoch: 019, Train: 0.9000, Val: 0.6960, Test: 

# Classification task on Cora dataset with CustomGATConv(meanFunc)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = CustomGATConv(in_channels, 8, heads=8, dropout=0.6, custom_func=meanFunc)
        self.conv2 = CustomGATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6, custom_func=meanFunc)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2500, Val: 0.1440, Test: 0.1300
Epoch: 002, Train: 0.2429, Val: 0.1280, Test: 0.1210
Epoch: 003, Train: 0.2286, Val: 0.1220, Test: 0.1170
Epoch: 004, Train: 0.3000, Val: 0.1700, Test: 0.1530
Epoch: 005, Train: 0.4500, Val: 0.3000, Test: 0.2840
Epoch: 006, Train: 0.5714, Val: 0.4000, Test: 0.4060
Epoch: 007, Train: 0.6714, Val: 0.4960, Test: 0.5030
Epoch: 008, Train: 0.7643, Val: 0.5820, Test: 0.6040
Epoch: 009, Train: 0.7929, Val: 0.6120, Test: 0.6200
Epoch: 010, Train: 0.7929, Val: 0.6040, Test: 0.6110
Epoch: 011, Train: 0.8071, Val: 0.6020, Test: 0.6120
Epoch: 012, Train: 0.8286, Val: 0.6360, Test: 0.6370
Epoch: 013, Train: 0.8643, Val: 0.6740, Test: 0.6660
Epoch: 014, Train: 0.8857, Val: 0.7300, Test: 0.7010
Epoch: 015, Train: 0.9000, Val: 0.7660, Test: 0.7430
Epoch: 016, Train: 0.9071, Val: 0.7820, Test: 0.7550
Epoch: 017, Train: 0.9214, Val: 0.7820, Test: 0.7600
Epoch: 018, Train: 0.9071, Val: 0.7560, Test: 0.7530
Epoch: 019, Train: 0.9214, Val: 0.7360, Test: 

# Classification task on Cora dataset with CustomGATConv(hadamardFunc)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = CustomGATConv(in_channels, 8, heads=8, dropout=0.6, custom_func=hadamardFunc)
        self.conv2 = CustomGATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6, custom_func=hadamardFunc)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.3786, Val: 0.2440, Test: 0.2630
Epoch: 002, Train: 0.5286, Val: 0.3400, Test: 0.3440
Epoch: 003, Train: 0.6429, Val: 0.4120, Test: 0.4250
Epoch: 004, Train: 0.6929, Val: 0.4700, Test: 0.4740
Epoch: 005, Train: 0.6714, Val: 0.4880, Test: 0.4880
Epoch: 006, Train: 0.6929, Val: 0.4640, Test: 0.4960
Epoch: 007, Train: 0.7429, Val: 0.4780, Test: 0.5060
Epoch: 008, Train: 0.8071, Val: 0.5420, Test: 0.5620
Epoch: 009, Train: 0.8071, Val: 0.5220, Test: 0.5470
Epoch: 010, Train: 0.8143, Val: 0.5420, Test: 0.5520
Epoch: 011, Train: 0.8429, Val: 0.5660, Test: 0.5760
Epoch: 012, Train: 0.8500, Val: 0.5840, Test: 0.5930
Epoch: 013, Train: 0.8571, Val: 0.6120, Test: 0.6080
Epoch: 014, Train: 0.8643, Val: 0.6340, Test: 0.6530
Epoch: 015, Train: 0.8714, Val: 0.6420, Test: 0.6660
Epoch: 016, Train: 0.9071, Val: 0.6580, Test: 0.6780
Epoch: 017, Train: 0.9071, Val: 0.6940, Test: 0.6930
Epoch: 018, Train: 0.9143, Val: 0.7000, Test: 0.7050
Epoch: 019, Train: 0.9143, Val: 0.7260, Test: 

# GATConv with concat Function

In [2]:
from typing import Optional, Tuple, Union, Any

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Parameter
from torch_sparse import SparseTensor, set_diag

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import (Adj, NoneType, OptPairTensor, OptTensor,
                                    Size)
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax

from torch_geometric.nn.inits import glorot, zeros


class CustomGATConv(MessagePassing):

    _alpha: OptTensor

    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        batch_dim: int = None,
        heads: int = 1,
        concat: bool = True,
        negative_slope: float = 0.2,
        dropout: float = 0.0,
        add_self_loops: bool = True,
        edge_dim: Optional[int] = None,
        fill_value: Union[float, Tensor, str] = 'mean',
        bias: bool = True,
        custom_func: Optional[Any] = lambda x,y: (x,y),
        **kwargs,
    ):
        kwargs.setdefault('aggr', 'add')
        super().__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.add_self_loops = add_self_loops
        self.edge_dim = edge_dim
        self.fill_value = fill_value
        self.custom_func = custom_func
        self.batch_dim = batch_dim

        # In case we are operating in bipartite graphs, we apply separate
        # transformations 'lin_src' and 'lin_dst' to source and target nodes:
        if isinstance(in_channels, int):
            self.lin_src = Linear(in_channels, heads * out_channels,
                                  bias=False, weight_initializer='glorot')
            self.lin_dst = self.lin_src
        else:
            self.lin_src = Linear(in_channels[0], heads * out_channels, False,
                                  weight_initializer='glorot')
            self.lin_dst = Linear(in_channels[1], heads * out_channels, False,
                                  weight_initializer='glorot')

        if batch_dim is not None:
          self.lin_batch_reduce = Linear(batch_dim, int(batch_dim/2), bias=False, weight_initializer='glorot')

        # The learnable parameters to compute attention coefficients:
        self.att_src = Parameter(torch.Tensor(1, heads, out_channels))
        self.att_dst = Parameter(torch.Tensor(1, heads, out_channels))

        if edge_dim is not None:
            self.lin_edge = Linear(edge_dim, heads * out_channels, bias=False,
                                   weight_initializer='glorot')
            self.att_edge = Parameter(torch.Tensor(1, heads, out_channels))
        else:
            self.lin_edge = None
            self.register_parameter('att_edge', None)

        if bias and concat:
            self.bias = Parameter(torch.Tensor(heads * out_channels))
        elif bias and not concat:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self._alpha = None

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_src.reset_parameters()
        self.lin_dst.reset_parameters()
        if self.lin_edge is not None:
            self.lin_edge.reset_parameters()
        glorot(self.att_src)
        glorot(self.att_dst)
        glorot(self.att_edge)
        zeros(self.bias)

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                edge_attr: OptTensor = None, size: Size = None,
                return_attention_weights=None):


        H, C = self.heads, self.out_channels

        # We first transform the input node features. If a tuple is passed, we
        # transform source and target node features via separate weights:
        if isinstance(x, Tensor):
            assert x.dim() == 2, "Static graphs not supported in 'GATConv'"
            x_src = x_dst = self.lin_src(x).view(-1, H, C)
        else:  # Tuple of source and target node features:
            x_src, x_dst = x
            assert x_src.dim() == 2, "Static graphs not supported in 'GATConv'"
            x_src = self.lin_src(x_src).view(-1, H, C)
            if x_dst is not None:
                x_dst = self.lin_dst(x_dst).view(-1, H, C)

        x = (x_src, x_dst)

        # Next, we compute node-level attention coefficients, both for source
        # and target nodes (if present):
        if self.batch_dim is None:
          alpha_src = (x_src * self.att_src).sum(dim=-1)
          alpha_dst = None if x_dst is None else (x_dst * self.att_dst).sum(-1)
          alpha = self.custom_func(alpha_src, alpha_dst)
        else:
          alpha_src = (x_src * self.att_src).sum(dim=-1).transpose(0, -1)
          alpha_src = self.lin_batch_reduce(alpha_src).transpose(-1, 0)
          alpha_dst = None if x_dst is None else (x_dst * self.att_dst).sum(-1).transpose(0, -1)
          if alpha_dst is not None: alpha_dst = self.lin_batch_reduce(alpha_dst).transpose(-1, 0)
          alpha = self.custom_func(alpha_src, alpha_dst)

        if self.add_self_loops:
            if isinstance(edge_index, Tensor):
                # We only want to add self-loops for nodes that appear both as
                # source and target nodes:
                num_nodes = x_src.size(0)
                if x_dst is not None:
                    num_nodes = min(num_nodes, x_dst.size(0))
                num_nodes = min(size) if size is not None else num_nodes
                edge_index, edge_attr = remove_self_loops(
                    edge_index, edge_attr)
                edge_index, edge_attr = add_self_loops(
                    edge_index, edge_attr, fill_value=self.fill_value,
                    num_nodes=num_nodes)
            elif isinstance(edge_index, SparseTensor):
                if self.edge_dim is None:
                    edge_index = set_diag(edge_index)
                else:
                    raise NotImplementedError(
                        "The usage of 'edge_attr' and 'add_self_loops' "
                        "simultaneously is currently not yet supported for "
                        "'edge_index' in a 'SparseTensor' form")

        # propagate_type: (x: OptPairTensor, alpha: OptPairTensor, edge_attr: OptTensor)  # noqa
        out = self.propagate(edge_index, x=x, alpha=alpha, edge_attr=edge_attr,
                             size=size)

        alpha = self._alpha
        assert alpha is not None
        self._alpha = None

        if self.concat:
            out = out.view(-1, self.heads * self.out_channels)
        else:
            out = out.mean(dim=1)

        if self.bias is not None:
            out += self.bias

        if isinstance(return_attention_weights, bool):
            if isinstance(edge_index, Tensor):
                return out, (edge_index, alpha)
            elif isinstance(edge_index, SparseTensor):
                return out, edge_index.set_value(alpha, layout='coo')
        else:
            return out

    def message(self, x_j: Tensor, alpha_j: Tensor, alpha_i: OptTensor,
                edge_attr: OptTensor, index: Tensor, ptr: OptTensor,
                size_i: Optional[int]) -> Tensor:
        # Given edge-level attention coefficients for source and target nodes,
        # we simply need to sum them up to "emulate" concatenation:
        alpha = alpha_j if alpha_i is None else alpha_j + alpha_i

        if edge_attr is not None:
            if edge_attr.dim() == 1:
                edge_attr = edge_attr.view(-1, 1)
            assert self.lin_edge is not None
            edge_attr = self.lin_edge(edge_attr)
            edge_attr = edge_attr.view(-1, self.heads, self.out_channels)
            alpha_edge = (edge_attr * self.att_edge).sum(dim=-1)
            alpha = alpha + alpha_edge

        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, index, ptr, size_i)
        self._alpha = alpha  # Save for later use.
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        return x_j * alpha.unsqueeze(-1)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, heads={self.heads})')

In [5]:
minFunc = lambda a, b : torch.minimum(a, b)
maxFunc = lambda a, b : torch.maximum(a, b)
sumFunc = lambda a, b : a + b
meanFunc = lambda a, b : (a + b) / 2
hadamardFunc = lambda a, b : a * b
concatFunc = lambda a, b: torch.cat((a, b))



In [6]:
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = CustomGATConv(in_channels, 8, dataset[0].x.shape[0], heads=8, dropout=0.6, custom_func=concatFunc)
        self.conv2 = CustomGATConv(8 * 8, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.1714, Val: 0.1000, Test: 0.1320
Epoch: 002, Train: 0.1714, Val: 0.0960, Test: 0.1210
Epoch: 003, Train: 0.1857, Val: 0.1080, Test: 0.1260
Epoch: 004, Train: 0.2071, Val: 0.1240, Test: 0.1380
Epoch: 005, Train: 0.2429, Val: 0.1360, Test: 0.1540
Epoch: 006, Train: 0.2929, Val: 0.1480, Test: 0.1720
Epoch: 007, Train: 0.3143, Val: 0.1720, Test: 0.1960
Epoch: 008, Train: 0.3214, Val: 0.1900, Test: 0.2100
Epoch: 009, Train: 0.3571, Val: 0.2260, Test: 0.2270
Epoch: 010, Train: 0.3714, Val: 0.2460, Test: 0.2490
Epoch: 011, Train: 0.4000, Val: 0.2740, Test: 0.2760
Epoch: 012, Train: 0.4571, Val: 0.2960, Test: 0.2990
Epoch: 013, Train: 0.5000, Val: 0.3340, Test: 0.3320
Epoch: 014, Train: 0.5286, Val: 0.3540, Test: 0.3780
Epoch: 015, Train: 0.5786, Val: 0.3880, Test: 0.4050
Epoch: 016, Train: 0.6000, Val: 0.4280, Test: 0.4240
Epoch: 017, Train: 0.6429, Val: 0.4580, Test: 0.4610
Epoch: 018, Train: 0.6929, Val: 0.4940, Test: 0.4920
Epoch: 019, Train: 0.7429, Val: 0.5260, Test: 

KeyboardInterrupt: ignored